여기에서는 기존의 머신러닝에서는 안되지만, __딥 러닝__에서는 되는 것을 살펴보자.

In [30]:
### XOR Problem with Machine learing
# 여기에서는 ML로는 XOR문제를 해결 할 수 없다는 한계를 확인 할 수 있다.
import tensorflow as tf


## training data set
x_data = [[0, 0], [0, 1], [1, 0], [1, 1]]
y_data = [[0], [1], [1], [0]]


## Placeholder
X = tf.placeholder(shape = [None, 2], dtype = tf.float32) # 2개 입력받는다.
Y = tf.placeholder(shape = [None, 1], dtype = tf.float32) # 1개의 결과 도출된다.


## Weight & bias
W = tf.Variable(tf.random_normal([2,1]), name = "weight") # 2개의 데이터를 2곳에서 입력받는다.
b = tf.Variable(tf.random_normal([1]), name = "bias") # 결과를 1개 도출한다.

In [31]:
## Hypothesis
logits = tf.matmul(X, W) + b
H = tf.sigmoid(logits)

## cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y))


## train
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

## Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## 학습
for step in range(30000):
    tmp, cost_val = sess.run([train, cost], feed_dict = {X:x_data, Y:y_data})
    if step % 3000 == 0:
        print("cost : {}".format(cost_val))

cost : 0.7837026119232178
cost : 0.6931471824645996
cost : 0.6931471824645996
cost : 0.6931471824645996
cost : 0.6931471824645996
cost : 0.6931471824645996
cost : 0.6931471824645996
cost : 0.6931471824645996
cost : 0.6931471824645996
cost : 0.6931471824645996


In [32]:
## 정확도 측정
predict = tf.cast(H>0.5, dtype=tf.float32)
correct = tf.equal(predict, Y)
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print("correct rate : {}".format(sess.run(accuracy, feed_dict={X:x_data, Y:y_data})))

correct rate : 0.25


correct rate가 0.25, 학습이 되지 않았다는 것을 알 수 있다.
- - -
이제 Deep learing을 이용해 XOR 문제를 해결해 보자.

In [33]:
# XOR with Deep learing
import tensorflow as tf

## 0. 그래프 초기화하기 (딥러닝은 그래프 초기화 필요/ 중복선언 예방)
tf.reset_default_graph()


## training data set
x_data = [[0, 0], [0, 1], [1, 0], [1, 1]]
y_data = [[0], [1], [1], [0]]


## Placeholder
X = tf.placeholder(shape = [None, 2], dtype = tf.float32)
Y = tf.placeholder(shape = [None, 1], dtype = tf.float32)


## Weight & bias
W1 = tf.Variable(tf.random_normal([2,2]), name = "weight1") # 앞 행렬값은 입력데이터의 column의 수이고, 뒷 행렬값은 계산 횟수이다.
b1 = tf.Variable(tf.random_normal([2]), name = "bias1") # 총 계산 횟수이다. OUT put의 개수가 된다.
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1) # 첫번째 layer의 결과값이다. [2]개가 도출된다.

W2 = tf.Variable(tf.random_normal([2,1]), name = "weight2") # 앞 숫자는 인풋값이다. 뒷 숫자는 아웃풋값이다.
#  -> 위의 layer의 아웃풋이 3개였으면 [3, x] 이 되었을 것. 앞 레이어의 output = 이번 layer의 input 
#  -> 이후에 레이어가 또 있었다면, 아웃풋 값이 1이 아니었을수 도 있다. but. 우리는 layer2에서 결론을 내야 하기에 [x, 1]형태가 되었다.
b2 = tf.Variable(tf.random_normal([1]), name = "bias2") # 결과값을 1개 도출하기에 output 1개

- - -
만약에 알고리즘을 심화 시키고 싶어서 layer를 확장하고 싶다면,    
Deep learing에서는 레이어 수를 편하게 늘려 줄 수 있다.

위에서 나온 1st layer의 아웃풋을 10개로 늘려주고 두번째 레이어의 인풋을 10개로 늘려주면,    
1번째 레이어에서 수행하는 rogistic F가 10개로 늘어난다.    
훨씬 더 심화된 학습을 시킬 수 있다.

__Layer를 확장하고 싶다면, Weight & BIas부분을 이렇게 바꾸면 된다.__
```
W3 = tf.Variable(tf.random_normal([2,10]), name = "weight3") # 입력데이터의 column의 수와 같아야
b3 = tf.Variable(tf.random_normal([10]), name = "bias3")
layer3 = tf.sigmoid(tf.matmul(X, W3) + b3) # 첫번째 layer의 결과값이다. [2]개가 도출된다.

W4 = tf.Variable(tf.random_normal([10,1]), name = "weight4") # 앞 숫자는 인풋값이다. 뒷 숫자는 아웃풋값이다.
b4 = tf.Variable(tf.random_normal([1]), name = "bias4") # 결과값을 1개 도출하기에 output 1개
```
___이 때 제일 중요한 점은 Input data와 Output데이터의 크기를 맞추어주는 것이다.___


위 의 예시처럼 layer를 늘린다면 정확도는 훨씬 높아 질 수 있지만, 계산은 기하급수적으로 많아진다.    
XOR문제를 해결하고 나서, 이전에 학습했던 MNIST를 이용해 multi layer를 실습해보자.
- - -

In [34]:
## Hypothesis
logit = tf.matmul(layer1, W2) + b2
H = tf.sigmoid(logit)


## cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=Y))


## train
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

## Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## 학습
for step in range(30000):
    tmp, cost_val = sess.run([train, cost], feed_dict = {X:x_data, Y:y_data})
    if step % 3000 == 0:
        print("cost : {}".format(cost_val))


## 정확도 측정
predict = tf.cast(H>0.5, dtype=tf.float32)
correct = tf.equal(predict, Y)
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print("\n correct rate : {}".format(sess.run(accuracy, feed_dict={X:x_data, Y:y_data})))

cost : 0.6967978477478027
cost : 0.5093070268630981
cost : 0.24782325327396393
cost : 0.04813949763774872
cost : 0.026779120787978172
cost : 0.018456988036632538
cost : 0.014052731916308403
cost : 0.011333964765071869
cost : 0.009491097182035446
cost : 0.008160640485584736

 correct rate : 1.0


XOR 문제가 성공적으로 학습 되었다는 것을 알 수 있다.
- - -
이제 위에서 말한 것처럼 다수의 Layer에 다수의 Node를 만들어 학습시켜보자.

In [35]:
# MNIST with Deep learing
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## 0. 그래프 초기화하기 (딥러닝은 그래프 초기화 필요/ 중복선언 예방)
tf.reset_default_graph()


## loading data set
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

## Placeholder
X = tf.placeholder(shape = [None, 784], dtype = tf.float32)
Y = tf.placeholder(shape = [None, 10], dtype = tf.float32)


## Weight & bias
W1 = tf.Variable(tf.random_normal([784,256]), name = "weight1")
b1 = tf.Variable(tf.random_normal([256]), name = "bias1")
layer1 = tf.sigmoid(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([256,512]), name = "weight2")
b2 = tf.Variable(tf.random_normal([512]), name = "bias2")
layer2 = tf.sigmoid(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([512,10]), name = "weight3") # 앞 숫자는 인풋값이다. 뒷 숫자는 아웃풋값이다.
b3 = tf.Variable(tf.random_normal([10]), name = "bias3") # 결과값을 1개 도출하기에 output 1개

## Hypothesis
logit = tf.matmul(layer2, W3) + b3
H = tf.sigmoid(logit)


## cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=Y))


## train
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

## Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())


## 학습
train_epoch = 50 # 학습을 몇번 할 것인가.
batch_size = 100 # 55000의 데이터를 한번에 학습하기가 쉽지 않다. 그래서 한번의 epoch때에도 학습을 나누어서 한다. 100개씩 550번 학습

for step in range(train_epoch) :    
    num_of_iter = int(mnist.train.num_examples/batch_size) # 여기에서는 550이다.
    
    for i in range(num_of_iter) :
        batch_x, batch_y = mnist.train.next_batch(batch_size) # mnist에서는 배치크기에 따라 데이터들을 자동으로 만들어주는 함수가 있다.
        
        tmp_train, cost_val = sess.run([train, cost],
                                      feed_dict = {X: batch_x, Y : batch_y})

    if step % 3 == 0:
        print("cost 값은 {}".format(cost_val))

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz
cost 값은 0.9669561386108398
cost 값은 0.7627556324005127
cost 값은 0.705893337726593
cost 값은 0.43712612986564636
cost 값은 0.45274674892425537
cost 값은 0.3598247766494751
cost 값은 0.33320873975753784
cost 값은 0.32405734062194824
cost 값은 0.3738652467727661
cost 값은 0.22693875432014465
cost 값은 0.25491929054260254
cost 값은 0.28238916397094727
cost 값은 0.19180917739868164
cost 값은 0.2083209604024887
cost 값은 0.20542992651462555
cost 값은 0.23687709867954254
cost 값은 0.1935681402683258


In [37]:
## 정확도 측정
predict = tf.cast(H>0.5, dtype=tf.float32)
correct = tf.equal(predict, Y)
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print("\n correct rate : {}".format(sess.run(accuracy, feed_dict={X: batch_x, Y : batch_y})))


 correct rate : 0.9290000200271606


이전의 machine learing 때 정확도가 겨우 90이었던 데 비해, 높아진 것을 알 수 있다.    
이번에는 딥러닝의 정확도를 향상시키는 여러 방법들을 알아보자.
- - -
### 알고리즘 변화

__Sigmoid__ 알고리즘은 나오는 값을 0~1 사이로 바꾸어준다.    
때문에 sigmoid 알고리즘을 계속 쓰다보면 숫자가 점점 더 소수화 되어가고, 결국 값이 점점 소멸되어 간다.    
propagation에서는 괜찮지만 __backpropagation에서 소멸__되는 현상 발생.    

원칙상으로는 sigmoid가 맞지만 sigmoid의 특성이 한계가 되어 새로운 함수를 찾았고, Re-lu 알고리즘을 사용하기 시작.    
Re-lu는 0이상의 값에서 x=y를 사용하는 알고리즘이다.    
때문에 단계가 많아지더라도 값이 소멸되지 않고, 유지 될 수 있다.

### 초기값 변화
초기 W값을 0, random을 사용 할 때에는 운이좋으면 학습이 잘 되지만, 운이 좋지 않을 때에는 학습이 잘 되지 않았다.    
그래서 초기값을 최적으롯 설정하는 여러 이론들이 나왔는데 대표적으로 Xavier initialization과 He's initialization이 있다.    
이번에는 Xavier initialization를 사용 해 보도록 하자.
_ _ _
이번에는 MNIST의 데이터를 분석 할 때 Re_lu알고리즘과 Xavier initializatioin을 사용 해 보도록 하자.

In [43]:
# MNIST with Deep learing 
# Using Re_lu Function / Xavier initializatioin
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## 0. 그래프 초기화하기 (딥러닝은 그래프 초기화 필요/ 중복선언 예방)
tf.reset_default_graph()


## loading data set
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

## Placeholder
X = tf.placeholder(shape = [None, 784], dtype = tf.float32)
Y = tf.placeholder(shape = [None, 10], dtype = tf.float32)


## Weight & bias
W1 = tf.get_variable("weight1", shape = ([784,256]),
                initializer = tf.contrib.layers.xavier_initializer()) # xavier_initializer 사용
b1 = tf.Variable(tf.random_normal([256]), name = "bias1")
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1) # Relu 알고리즘 사용

W2 = tf.get_variable("weight2", shape = ([256,512]),
                initializer=tf.contrib.layers.xavier_initializer()) # xavier_initializer 사용
b2 = tf.Variable(tf.random_normal([512]), name = "bias2")
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2) # Relu 알고리즘 사용

W3 = tf.get_variable("weight3", shape = ([512,10]),
                initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]), name = "bias3")

## Hypothesis
logit = tf.matmul(layer2, W3) + b3
H = tf.nn.relu(logit) # 이번에는 relu 알고리즘을 사용한다.


## cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=Y))


## train
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)

## Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())


## 학습
train_epoch = 50 # 학습을 몇번 할 것인가.
batch_size = 100 # 55000의 데이터를 한번에 학습하기가 쉽지 않다. 그래서 한번의 epoch때에도 학습을 나누어서 한다. 100개씩 550번 학습

for step in range(train_epoch) :    
    num_of_iter = int(mnist.train.num_examples/batch_size) # 여기에서는 550이다.
    
    for i in range(num_of_iter) :
        batch_x, batch_y = mnist.train.next_batch(batch_size) # mnist에서는 배치크기에 따라 데이터들을 자동으로 만들어주는 함수가 있다.
        
        tmp_train, cost_val = sess.run([train, cost],
                                      feed_dict = {X: batch_x, Y : batch_y})

    if step % 3 == 0:
        print("cost 값은 {}".format(cost_val))

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz
cost 값은 0.28176507353782654
cost 값은 0.20361237227916718
cost 값은 0.12640143930912018
cost 값은 0.11413269490003586
cost 값은 0.09745970368385315
cost 값은 0.10534271597862244
cost 값은 0.10097570717334747
cost 값은 0.0893247202038765
cost 값은 0.08723349124193192
cost 값은 0.09997054189443588
cost 값은 0.06449788063764572
cost 값은 0.05096662789583206
cost 값은 0.10295236110687256
cost 값은 0.06549856066703796
cost 값은 0.07569726556539536
cost 값은 0.0689300000667572
cost 값은 0.08535070717334747


In [44]:
## 정확도 측정
predict = tf.cast(H>0.5, dtype=tf.float32)
correct = tf.equal(predict, Y)
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print("\nRelu & xavier_initializer \n correct rate : {}".format(sess.run(accuracy, feed_dict={X: batch_x, Y : batch_y})))


Relu & xavier_initializer 
 correct rate : 0.9810000061988831


알고리즘과 초기값 설정을 바꿔주었더니, 정확도가 상당히 많이 올라갔다는 것을 알 수 있다. 
- - -

이렇게 다양한 이론들 덕분에, (Relu를 이용한)logistic의 수를 늘리고, 다양한 layer를 늘리기 시작했다.    
그러자 Over fitting이 일어나기 시작했다.
학습률은 올라갈 수 있지만, 

__오버피팅이 발생할 확률을 줄이기 위해서는__
1. 데이터 수가 많아야 한다.    
데이터가 작으면 딱 그 데이터에만 맞게 학습이 된다.
2. 불필요한 칼럼이나 속성을 줄여야 한다.    
칼럼을 없애거나, 합쳐서 칼럼의 수를 줄여야 한다. EDA가 중요하다.
3. 연산을 할 때, Random하게 일정 노드를 제외한다. (Drop out)    
매 학습 때마다 랜덤하게 일정 수의 노드를 제외함으로써 Overfitting을 제외한다. (수학적으로 증명 됨)
4. Regularization    
Weight에 너무 많은 값이 할당되지 않도록 한다.

- - -
우리의 학습은 Test data set의 acc가 상당히 높아서 Overfitting이 일어나지 않았다.    
하지만, 학습을 위해서 Drop-out을 해보자.

In [51]:
# MNIST with Deep learing 
# Using Re_lu Function / Xavier initializatioin
# Using Drop-out
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## 0. 그래프 초기화
tf.reset_default_graph()


## loading data set
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)


## Placeholder
X = tf.placeholder(shape = [None, 784], dtype = tf.float32)
Y = tf.placeholder(shape = [None, 10], dtype = tf.float32)


## Weight & bias
W1 = tf.get_variable("weight1", shape = ([784,256]),
                initializer = tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]), name = "bias1")
_layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
layer1 = tf.nn.dropout(_layer1, rate=0.3) # Drop out 실행

W2 = tf.get_variable("weight2", shape = ([256,512]),
                initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]), name = "bias2")
_layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
layer2 = tf.nn.dropout(_layer2, rate=0.3)  # Drop out 실행

W3 = tf.get_variable("weight3", shape = ([512,10]),
                initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]), name = "bias3")


## Hypothesis
logit = tf.matmul(layer2, W3) + b3  # 마지막 layer은 Drop out을 실행하지 않았다.
H = tf.nn.relu(logit)


## cost
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=Y))


## train
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)


## Session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())


## 학습
train_epoch = 50 # 학습을 횟수
batch_size = 100 # 데이터를 나눠서 학습하는 것

for step in range(train_epoch) :    
    num_of_iter = int(mnist.train.num_examples/batch_size) # 여기에서는 550이다.
    cost_val=0
    
    for i in range(num_of_iter) :
        batch_x, batch_y = mnist.train.next_batch(batch_size) # mnist에서는 배치크기에 따라 데이터들을 자동으로 만들어주는 함수가 있다.
        
        tmp_train, cost_val = sess.run([train, cost],
                                      feed_dict = {X: batch_x, Y : batch_y})

    if step % 3 == 0:
        print("cost 값은 {}".format(cost_val))
        
## 정확도 측정
predict = tf.cast(H>0.5, dtype=tf.float32)
correct = tf.equal(predict, Y)
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print("\nRelu & xavier_initializer & Drop out\n correct rate : {}".format(sess.run(accuracy, feed_dict={X: batch_x, Y : batch_y})))

Extracting ./data/mnist\train-images-idx3-ubyte.gz
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz
cost 값은 0.32603195309638977
cost 값은 0.21736404299736023
cost 값은 0.18691548705101013
cost 값은 0.15417160093784332
cost 값은 0.1457023024559021
cost 값은 0.12940064072608948
cost 값은 0.13325637578964233
cost 값은 0.11611662060022354
cost 값은 0.09814773499965668
cost 값은 0.10121145099401474
cost 값은 0.09383891522884369
cost 값은 0.08103030174970627
cost 값은 0.07991203665733337
cost 값은 0.07652615010738373
cost 값은 0.07895088195800781
cost 값은 0.07750395685434341
cost 값은 0.1158795952796936

Relu & xavier_initializer & Drop out
 correct rate : 0.972000002861023


과적합이 아니었기에 Drop out을 적용했더니 오히려 점수가 떨어졌다.

- - -
딥러닝이 발전하면서 여러가지로 세분화 되었다.

Deep learning    
-> CNN : 이미지 처리에 특화 (Converusion Neural Network)    
Deep learing     
-> RNN : 자연어 처리 (Recursive Neural Network)    
(자연어는 - 앞에 있는 단어에 뒤에 있는 단어에 영향을 끼친다.)

이렇게 두가지 알고리즘이 대표적인데, 다양한 알고리즘이 탄생하였고 지금도 생겨나고 있다.

- - -
지금도 계속 알고리즘이 생겨나고 있다.
이미지 처리도 지금은 CNN보다 좋은 알고리즘으로 GNN 등이 나타나고 있다.
이미지 처리를 잘 하기 위해서는 꼭 OpenCV를 같이 공부/사용 해야한다.
OpenCv는 이미지 처리에 특화 된 라이브러리

- - -
이번엔 CNN을 사용해서 MNIST를 학습시켜보자.

지금까지 우리가 한 학습은 Fully connected 네트워크이다. (모든 노드들이 연결되어 있었다.)    
CNN 또한 뒤쪽에 FC 단계가 있다. 하지만 CNN의 중요한 특징은 FC 앞쪽에 있는 새로운 계층들이다.    

CNN의 앞 부분은,    
이미지 전체를 학습하는 것이 아니라 이미지를 잘게 쪼개서, 각 부분들을 학습한다.    
그 잘게 쪼개진 이미지들을 학습하는 과정에서 이미지의 여러 ___특징___들이 학습된다.    

잘게 쪼개진 이미지들의 용량이 커버리면 전체 용량이 너무 커져버린다.
때문에 크기를 줄여줘야 원할한 학습이 가능해진다.    

- - -
## CNN

### Filter
필터의 크기는 내가 마음대로 설정 할 수 있지만 Depth는 원본 이미지(3)와 같아야한다. -> 원본 이미지와 매칭해야 되기 때문    
Filter값을 정하고 Stride를 정하면 원본 이미지와 계산되어 몇개의 특징을 Node가 도출될지 계산된다.    

### Stride
Stride는 Image의 크기와 Filter값에 의해 가능한 숫자가 달라진다. (Default = 1) 
> ex>    
크기가 7 * 7 * 3 인 이미지가 있다. (depth가 3인 이유 : 이미지는 ( R, G, B ) 3개의 값이 1 pixel을 구성하기 때문)    
크기가 3 * 3 * 3 인 필터를 적용한다면, stride가 ~가 가능하다.    
Stride가 Default인 1이라고 가정하면, 
    (image)7 * 7 * 3 -(filter : 3 * 3 * 3)-> (image)5 * 5 * 3 가 된다.

이렇게 필터를 한단계 더 적용하게 되면 이미지 크기가 3 * 3 * 3이 되는데, 이는 크기가 너무 작아서 특징을 추출하기 어려워진다.
그래서 __Padding__이 등장했다.

### Padding



그리고 이러한 특징들이 적용된 필터들을 하나의 계층에서도 여러개 만들 수 있다는 것이다.    
그렇게 되면 점점 더 많은 필터들이 중첩되기 시작한다.
> ex>
첫 이미지    CONV, ReLU    Layer 1
 32x32x3   --(5x5x3, 6)--> 28x28x6


이런식으로 진행되면 최종적으로,
이미지의 크기는 줄어들지만 훨씬 더 많은 특징들을 학습 하게 된다.

### multi filter
여러개의 필터를 적용함으로써, 한 layer에서도 여러개의 필터를 적용 할 수 있다.
덕분에 Depth 가 점점 더 깊어지게 된다.

### Pooling (Max pooling)
이건 특징을 추출할수록 이미지의 크기가 너무 커져서, 특징은 유지하며 이미지 크기를 줄이는 단계이다.
여기는 필터와 비슷한 개념인 커널을 사용한다.
커널이 필터와 같이 stride값에 따라, 이미지를 부분 부분으로 자르면서 가장 큰 값의 특징을 추출한다.
그 덕분에 커널의 크기와 Stride 값에 따라 이미지의 특징을 잘 유지하면서도 이미지의 크기를 줄인 이미지를 만들 수 있다.

Pooling은 이미지 크기를 조정 하고 싶을 때, 언제든 사용 할 수 있다.

